In [1]:
import sys
sys.path.append('../src')

import os
import requests
import json
import pandas as pd
import betfairlightweight
from betfairlightweight import filters
from datetime import datetime, timedelta
import time

from morning_price import MorningPrice
from betfair import Betfair

pd.set_option('display.max_columns', None)

In [2]:
app_key = 'ABGJLOlKaLtTsMIp'

username1 = input("Email:")
password1 = input("Password:")

In [3]:
trading = betfairlightweight.APIClient(
        username1, password1, app_key
    )
trading.login_interactive()

trading.session_token

trading.keep_alive()

<KeepAliveResource>

In [4]:
# Set the start and end time to cover the entire day in UTC
today = datetime.utcnow()
start_time = today.replace(hour=0, minute=0, second=0, microsecond=0)
end_time = today.replace(hour=23, minute=59, second=59, microsecond=999999)

In [5]:
aa = MorningPrice(start_time, end_time)
results = aa.markets(start_time, end_time, trading)

In [6]:
for i in results:
        print(
            "{0} {1:d}:{2:02d} {3} ({4})".format(
                i.market_id,
                i.market_start_time.hour, 
                i.market_start_time.minute, 
                i.event.venue,  
                i.description.market_type,
            )
        )


1.233380084 12:47 Warwick (WIN)
1.233380142 12:55 Fontwell (WIN)
1.233379799 13:10 Beverley (WIN)
1.233380090 13:20 Warwick (WIN)
1.233380149 13:30 Fontwell (WIN)
1.233379806 13:45 Beverley (WIN)
1.233380097 13:55 Warwick (WIN)
1.233380156 14:05 Fontwell (WIN)
1.233379811 14:20 Beverley (WIN)
1.233380104 14:30 Warwick (WIN)
1.233380163 14:40 Fontwell (WIN)
1.233379816 14:55 Beverley (WIN)
1.233380118 15:05 Warwick (WIN)
1.233380189 15:10 Newcastle (WIN)
1.233380170 15:15 Fontwell (WIN)
1.233379943 15:30 Beverley (WIN)
1.233380111 15:40 Warwick (WIN)
1.233380196 15:45 Newcastle (WIN)
1.233380177 15:50 Fontwell (WIN)
1.233379950 16:02 Beverley (WIN)
1.233380125 16:10 Warwick (WIN)
1.233380202 16:20 Newcastle (WIN)
1.233380184 16:25 Fontwell (WIN)
1.233379957 16:37 Beverley (WIN)
1.233380131 16:45 Warwick (WIN)
1.233380209 16:55 Newcastle (WIN)
1.233380216 17:30 Newcastle (WIN)
1.233380223 18:00 Newcastle (WIN)
1.233380230 18:30 Newcastle (WIN)
1.233380237 19:00 Newcastle (WIN)
1.23338024

In [7]:
df1 = aa.market_runner(results)
df1

,market_id,market_start_time,venue,market_type,selection_id,horse_name
0,1.233380084,2024-09-24 12:47:00,Warwick,WIN,505010,Get With It
1,1.233380084,2024-09-24 12:47:00,Warwick,WIN,50862604,Ballinoulart
2,1.233380084,2024-09-24 12:47:00,Warwick,WIN,50307497,Smurfette
3,1.233380084,2024-09-24 12:47:00,Warwick,WIN,36715176,Guguss Collonges
4,1.233380084,2024-09-24 12:47:00,Warwick,WIN,38259504,Jony Max
...,...,...,...,...,...,...
296,1.233380244,2024-09-24 19:30:00,Newcastle,WIN,49526290,Noble Captain
297,1.233380244,2024-09-24 19:30:00,Newcastle,WIN,13123197,Tathmeen
298,1.233380244,2024-09-24 19:30:00,Newcastle,WIN,39081315,Howzak
299,1.233380244,2024-09-24 19:30:00,Newcastle,WIN,7072840,Quanah


# I need to get the a list of id's from the market_id column and then create a loop for the list_market_book 

In [8]:
market_id_list = aa.market_id_list(df1)

In [9]:
market_books1 = aa.market_books(market_id_list, trading)

In [10]:
df2 = aa.morning_price(market_books1)

# join df1 and df2

In [11]:
df3 = aa.join(df1, df2)

In [12]:
df3

,market_id,market_start_time,venue,market_type,selection_id,horse_name,status,morning_price,total_matched
0,1.233380084,2024-09-24 12:47:00,Warwick,WIN,505010,Get With It,ACTIVE,4.8,71482.97
1,1.233380084,2024-09-24 12:47:00,Warwick,WIN,50862604,Ballinoulart,ACTIVE,8.0,71482.97
2,1.233380084,2024-09-24 12:47:00,Warwick,WIN,50307497,Smurfette,ACTIVE,5.7,71482.97
3,1.233380084,2024-09-24 12:47:00,Warwick,WIN,36715176,Guguss Collonges,ACTIVE,8.8,71482.97
4,1.233380084,2024-09-24 12:47:00,Warwick,WIN,38259504,Jony Max,ACTIVE,23.0,71482.97
...,...,...,...,...,...,...,...,...,...
296,1.233380244,2024-09-24 19:30:00,Newcastle,WIN,49526290,Noble Captain,ACTIVE,29.0,5336.82
297,1.233380244,2024-09-24 19:30:00,Newcastle,WIN,13123197,Tathmeen,ACTIVE,30.0,5336.82
298,1.233380244,2024-09-24 19:30:00,Newcastle,WIN,39081315,Howzak,ACTIVE,36.0,5336.82
299,1.233380244,2024-09-24 19:30:00,Newcastle,WIN,7072840,Quanah,ACTIVE,95.0,5336.82


In [15]:
df4 = pd.read_csv('../data/daily/2024_09_24.csv')
df4.head(10)

,Race Time,Course,Distance (y),Race Type,Position,Horse,Weight (pounds),SP Odds Decimal,BF Decimal SP,"Won (1=Won, 0=Lost)",Proform Speed Rating,Evening Price,Breakfast Price,Morning Price,Won P/L Before
0,24/09/2024 1:40:00 PM,Beverley,2187,Turf,NaN,Another Beautiful,137,NaN,1,0,NaN,21.0,21.0,15.00,-8.00
1,24/09/2024 1:40:00 PM,Beverley,2187,Turf,NaN,Beguiling,129,NaN,1,0,NaN,34.0,34.0,34.00,-5.00
2,24/09/2024 1:40:00 PM,Beverley,2187,Turf,NaN,Dandy's Angel,130,NaN,1,0,NaN,8.0,9.0,7.00,-7.42
3,24/09/2024 1:40:00 PM,Beverley,2187,Turf,NaN,Dr Rio,138,NaN,1,0,NaN,5.0,5.0,5.50,-26.88
4,24/09/2024 1:40:00 PM,Beverley,2187,Turf,NaN,Etretat,137,NaN,1,0,NaN,5.5,5.5,8.00,-16.10
5,24/09/2024 1:40:00 PM,Beverley,2187,Turf,NaN,Jez Bomb,131,NaN,1,0,NaN,4.0,4.0,4.33,-3.25
6,24/09/2024 1:40:00 PM,Beverley,2187,Turf,NaN,Just Hiss,142,NaN,1,0,NaN,15.0,15.0,13.00,-23.75
7,24/09/2024 1:40:00 PM,Beverley,2187,Turf,NaN,Love Yours,134,NaN,1,0,NaN,8.0,7.0,8.00,-8.00
8,24/09/2024 1:40:00 PM,Beverley,2187,Turf,NaN,Olympicus,137,NaN,1,0,NaN,12.0,12.0,12.00,-3.00
9,24/09/2024 1:40:00 PM,Beverley,2187,Turf,NaN,Panama City,134,NaN,1,0,NaN,26.0,26.0,26.00,5.33
